<center>
    <h1>
        <b>
            Finding the Sweet Spot: Towards a Framework for Optimal Configurations of Feature Attribution Based Explanations
        </b>
    </h1>
<h2>
    Arthur-Louis Heath <br>
</h2>
    <i>
        <b>
            Created for BSc Thesis: Computer Science and Mathematics (2021) <br>
            Department of Computing Science<br>
            University of Aberdeen
        </b>
    </i>
</center>

# INSTALL DEPENDENCIES & LOAD LIBRARIES

In [1]:
###### INSTALL DEPENDENCIES #######
!python3 -m pip install numpy
!python3 -m pip install pandas
!python3 -m pip install sklearn
!python3 -m pip install scipy
!python3 -m pip install lime
!python3 -m pip install shap
!python3 -m pip install dice-ml
!python3 -m pip install jinja2
!python3 -m pip install num2words
!python3 -m pip install ipywidgets ipython notebook 


# Alternatively use this as required
#!python -m pip install numpy
#!python -m pip install pandas
#!python -m pip install sklearn
#!python -m pip install scipy
#!python -m pip install lime
#!python -m pip install shap
#!python -m pip install dice-ml
#!python -m pip install jinja2
#!python -m pip install num2words
#!python -m pip install ipywidgets ipython notebook 





In [1]:
###### LOAD LIBRARIES #######

#from dataclasses import dataclass

# Jupyter

from ipywidgets import interactive


# Data Processing

import numpy as np
import pandas as pd

from copy import deepcopy


# Model

import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import ConfusionMatrixDisplay as cmd

import scipy

from sklearn.preprocessing import LabelEncoder


# Explainability Tools

## LIME
import lime
import lime.lime_tabular

## SHAP
import shap

## Global Linear Approximation (Linear Regression)
from sklearn import linear_model

## DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions


# Natural Language Generation
from jinja2 import Template, FileSystemLoader, Environment
from num2words import num2words


# Visualisation
import matplotlib

from matplotlib import pyplot as plt
from matplotlib import cm, rc, gridspec

ImportError: No module named ipywidgets

# DATASET
## Analytics Vidhya Loan Approval Dataset
## Source: https://www.kaggle.com/leonbora/analytics-vidhya-loan-prediction

## Load Dataset

In [ ]:
# Load Dataset
orig_dataset = pd.read_csv('loan_appproval_dataset.csv')
orig_dataset = orig_dataset.drop(columns=['Loan_ID'])
orig_dataset


### Data Cleaning

In [ ]:
# Remove rows with missing values (EG LoanAmount = NaN)

orig_dataset = orig_dataset.dropna()

orig_dataset

In [ ]:
# Prettify feature names for display

pretty_names = ['Gender', 
                'Marrital Status',
                'Num. Dependents',
                'Education', 
                'Employment Type', 
                'Applicant Income', 
                'Co-Applicant Income', 
                'Loan Amount', 
                'Loan Duration', 
                'Credit History Status', 
                'Property Area Type', 
                'Application Status']

orig_dataset.columns = pretty_names

target_feat_id = 'Application Status'

orig_dataset.columns

In [ ]:
# Prettify label names for categorical feature values

# 'Marrital Status'
print(orig_dataset['Employment Type'].unique())
orig_dataset.loc[:, ['Employment Type']] = orig_dataset['Employment Type'].apply(lambda x: 'Self-Employed' if x == 'Yes' else 'Not Self-Employed' if x == 'No' else x)
print(orig_dataset['Employment Type'].unique())
print()

# 'Marrital Status'
print(orig_dataset['Marrital Status'].unique())
orig_dataset.loc[:, ['Marrital Status']] = orig_dataset['Marrital Status'].apply(lambda x: 'Married' if x == 'Yes' else 'Unmarried' if x == 'No' else x)
print(orig_dataset['Marrital Status'].unique())
print()

# 'Application Status'
print(orig_dataset['Application Status'].unique())
orig_dataset.loc[:, ['Application Status']] = orig_dataset['Application Status'].apply(lambda x: 'Approved' if x == 'Y' else 'Rejected' if x == 'N' else x)
print(orig_dataset['Application Status'].unique())
print()

# 'Accesible Credit History'
print(orig_dataset['Credit History Status'].unique())
orig_dataset.loc[:, ['Credit History Status']] = orig_dataset['Credit History Status'].apply(lambda x: 'Available' if x == 1 else 'Unavailable' if x == 0 else x)
print(orig_dataset['Credit History Status'].unique())


### Encode Categorical Data

In [ ]:
# Multi column data encoder class

# Required since sklearns default encoder cannot decode on multiple columns
# Adapted from code courtesy of user gereleth on stack overflow
# (source: https://stackoverflow.com/questions/58217005/how-to-reverse-label-encoder-from-sklearn-for-multiple-columns)

class MultiColumnLabelEncoder:

    def __init__(self, columns=None):
        self.columns = columns # array of column names to encode


    def fit(self, X, y=None):
        self.encoders = {}
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            self.encoders[col] = LabelEncoder().fit(X[col])
        return self


    def transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].transform(X[col])
        return output


    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)


    def inverse_transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].inverse_transform(X[col])
        return output

In [ ]:
# Encode categorical data

dataset = deepcopy(orig_dataset)

# List of categorical features to encode (as integers starting from 0)
label_encode = ['Gender', 
                'Marrital Status',
                'Num. Dependents',
                'Education', 
                'Employment Type',
                'Credit History Status', 
                'Property Area Type', 
                'Application Status']

le = MultiColumnLabelEncoder(columns=label_encode)

dataset = le.fit_transform(dataset)

dataset

In [ ]:
# Create test/train split for Model Training

X = dataset.drop(columns=[target_feat_id]).values
y = dataset[target_feat_id]

test_size = 0.3
random_state = 0

# Parameter random_state=0 ensures the version of the model is identical to the one used in the project. Feel free to change this. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)


# MODEL
## Create and train SKLearn random forrest classifier

In [ ]:

untrained_model = sklearn.ensemble.RandomForestClassifier(n_estimators=1000)
model = untrained_model.fit(X_train, y_train.astype('int'))

# Display classifier metrics
y_pred = [lab for lab in model.predict(X_test)]

test_class = [np.int64(z) for z in y_test.values]

report = classification_report(test_class, y_pred)
print('\n' + report)
accuracy = accuracy_score(test_class, y_pred)
print('Accuracy: ' + str(accuracy))

# DATA INSTANCE
## Select data instance and compute feature attributions of SHAP LIME and GAM libraries

In [ ]:
# Select data instance
data_instance_index = 62
data_instance = np.array(dataset.loc[data_instance_index])[:-1]
data_instance_orig_labels = orig_dataset.loc[data_instance_index]

# print(X_test[13])
print(data_instance)
print(data_instance_orig_labels)

# Create df to hold attribution results for each method and composites
feature_names = dataset.drop(columns=[target_feat_id]).columns

attributions = pd.DataFrame(columns = feature_names)

# Compute LIME attributions

In [ ]:
# list of indices (ints) corresponding to the categorical columns. Everything else will be considered continuous
categorical_feature_indecies =  [0, 1, 2, 3, 4, 8, 9, 10]

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train,
                                                   feature_names=list(feature_names), 
                                                   categorical_features=categorical_feature_indecies, 
                                                   verbose=True,
                                                   mode='classification')


In [ ]:
exp = explainer.explain_instance(data_instance, model.predict_proba, num_features=len(feature_names))

In [ ]:
# Convert back from lime representations to feature names
lime_attr = {}

# WARNING: Features who's names appear as substrings of names of other features break this code
for feat in exp.as_list():
    for f_name in list(feature_names):
        if(f_name in feat[0]):
            lime_attr[f_name] = feat[1]
            
lime_attr = pd.Series(lime_attr)
lime_attr

In [ ]:
# record lime attributions
lime_attr.name = 'LIME'

attributions = attributions.append(lime_attr)

attributions

In [ ]:
# Visualise lime attributions
# %matplotlib inline
fig = exp.as_pyplot_figure()

# Compute SHAP attributions

In [ ]:


pd.DataFrame(np.array([data_instance]), columns=feature_names)

In [ ]:
explainer = shap.TreeExplainer(model)

shap_values = explainer.shap_values(
    pd.DataFrame(np.array([data_instance]), 
                 columns=feature_names))

# Select appropriate set of shap values based on model prediction
shap_values = shap_values[model.predict([data_instance])[0]]

shap_values
            
#shap_values
# 
# shap.plots.force(shap_values)
# dict(pd.Series(shap_values, index=feature_names))
# dictionary = dict(zip(shap_values, feature_names))
# 
# shap.plots.bar({A: B for A, B in zip(feature_names, shap_values)})
# shap.plots.bar(dict(pd.Series(list(shap_values[0]), index=feature_names)))

In [ ]:
# record shap attributions

# list(shap_values[0])

shap_attr = pd.Series(list(shap_values[0]), index=feature_names)


shap_attr.name = 'SHAP'

#shap_attr
attributions = attributions.append(shap_attr)

type(attributions.index)

attributions

# Train general linear model for alternative global attribution estimator
### (this is disabled by default but can be enabled as needed)

In [ ]:
# Run linear regression

regr = linear_model.LinearRegression()

# Since we are explaining model, run regression on model predictions

# Warning, note that here we are using variable encodings as dummy variables, depending on feature semantics this may not be optimal

X_all = dataset.drop(columns=[target_feat_id]).to_numpy()

Y_all = model.predict(X_all)

regr.fit(X_all, Y_all)

print(regr.coef_)


In [ ]:
regr_attr = pd.Series(list(regr.coef_), index=feature_names)

regr_attr.name = 'GLOBAL REGRESSION'

# attributions = attributions.append(regr_attr)

attributions

# GENERATE COUNTERFACTUAL/S
## Using Dice-ML (Based on https://arxiv.org/abs/1905.07697)

In [ ]:
# Features that can vary in counterfactual examples
# 'Marrital Status', 'Num. Dependents'?

features_to_vary = ['Education', 
                    'Employment Type', 
                    'Applicant Income', 
                    'Co-Applicant Income',  
                    'Loan Duration',
                    'Credit History Status', 
                    'Property Area Type']

In [ ]:
# Wrap objects for use with dice
dice_data = dice_ml.Data(dataframe=dataset,
                 continuous_features=[x for x in dataset.columns if x not in label_encode],
                 outcome_name=target_feat_id)

dice_model = dice_ml.Model(model=model, backend="sklearn")

# KD-Tree finds counterfactuals within the dataset
# This is good since we are working with a rather innacurate model and small dataset
# resulting in randomly sampled counterfactuals being erratic

# dice_exp = dice_ml.Dice(dice_data, dice_model, method='kdtree')

dice_exp = dice_ml.Dice(dice_data, dice_model)
dice_instance = pd.DataFrame([data_instance], columns=feature_names)

# Generate counterfactuals
counterfactual = dice_exp.generate_counterfactuals(dice_instance, 
                                                   total_CFs=1, 
                                                   desired_class="opposite",
                                                   features_to_vary=features_to_vary)

# features_to_vary= //// Features to vary
# feature_weights=feature_weights /// Dict of feature weithgts

counterfactual.visualize_as_dataframe(show_only_changes=True)

# Through minor surgery we can obtain counterfactuals as a dataframe
counterfactual = counterfactual.cf_examples_list[0].final_cfs_df

counterfactual


In [ ]:
# Save/Load counterfactuals for reproducability

# This line can be uncommented to save newly generated counterfactuals
# counterfactual.to_csv('survey/counter_fact/counterfactuals_save.csv')

# By default, this file contains counterfactual examples used in the study
counterfactual = pd.read_csv('survey/counter_fact/counterfactuals_load.csv')

counterfactual = counterfactual.reset_index(drop=True)

counterfactual

In [ ]:
# Decode feature encodings for newly generated counterfactual example

# Fix column dtypes
counterfactual[label_encode] = counterfactual[label_encode].astype(int)

decoded_counterfactuals = le.inverse_transform(counterfactual)
# decoded_counterfactuals = le.inverse_transform(orig_)

decoded_counterfactuals

# Estimate attributions using DiCE

In [ ]:
# Compute dice attributions from set of counterfactuals
imp = dice_exp.local_feature_importance(dice_instance, total_CFs=50)

print(imp.local_importance[0])

In [ ]:
# Add dice attributions

dice_attr = pd.Series(imp.local_importance[0].values(), index=feature_names)

dice_attr.name = 'DICE-ML'

#shap_attr
attributions = attributions.append(dice_attr)

attributions

# NORMALISE ATTRIBUTIONS


In [ ]:
# Express attributions as percentage of the sum of the absolute value of each attribution
# This is done so that they can be presented on a common axis.

for index, row in attributions.iterrows():
    scale = row.abs().sum()
    row.update(row.apply(lambda x : x * 100 / scale))

attributions

# CALCULATE COMPOSITE SCORES

In [ ]:
composite_scores = {}
for col in attributions:
    composite_scores[col] = np.mean(attributions[col])

composite_scores = pd.Series(composite_scores)
composite_scores.name = 'COMPOSITE'

#shap_attr
attributions = attributions.append(composite_scores)

attributions 

# PREPARE TEXT TEMPLATES

In [ ]:
# Prepare Jinja2 templates for NLG

APP_OUT = "loan_approval_outcome.txt"
BASE_EXP = "base_textual_explanation.txt"
COUNTER = "textual_counterfactual.txt"


In [ ]:
templateLoader = FileSystemLoader(searchpath="./templates")
templateEnv = Environment(loader=templateLoader)

# Load templates
temp_app_outcome = templateEnv.get_template(APP_OUT)
temp_base_exp = templateEnv.get_template(BASE_EXP)
temp_counterfact = templateEnv.get_template(COUNTER)

temp_app_outcome.render(acceptance='Hello World!')

# CONSTRUCT EXPLANATION

In [ ]:
# Display settings

font = {'family' : 'DejaVu Sans',
        'weight' : 'bold',
        'size'   : 35}

rc('font', **font)
rc('text',usetex=True)
rc('text.latex', preamble=r'\usepackage{color}')

In [ ]:
# Structure to store data needed to generate explanation unrelated to explanation settings/layers

@dataclass
class PublicData:
    # Pandas dataframe of counterfactual explanations
    counterfactuals : pd.DataFrame
    
    # Pandas dataframe of feature attributions
    attributions : pd.DataFrame
        
    # Feature values of explanation instance
    feature_values : pd.Series

In [ ]:
# Initialise explanation data

pub_data = PublicData(
    counterfactuals=decoded_counterfactuals,
    attributions=attributions,
    feature_values = data_instance_orig_labels)


In [ ]:
# Helper function for cleaning data before display

def add_units(features):
    features = deepcopy(features)
    # Add units to select features and remove decimal places
    # features <-- pandas series of feature values
    features['Applicant Income'] = str(int(features['Applicant Income'])) + ' \$/Mo'
    features['Co-Applicant Income'] = str(int(features['Co-Applicant Income'])) + ' \$/Mo'
    features['Loan Amount'] = str(int(features['Loan Amount'])) + '000 \$'
    features['Loan Duration'] = str(int(features['Loan Duration'])) + ' Months'
    
    return features

print(add_units(pub_data.feature_values))

In [ ]:
def construct_explanation(pub_data, visualise=False, num_feats=2, display_min=False, display_composite=True, display_counterfactuals=False):
    '''
    Method to generate arbitrary explanation from configuration space.
    
    The parameters are as follows:
    visualise <-- Weather to display explanations as text or visualise as plots
    attributions <-- pandas dataframe containing normalised attributions of each model and composite attribution scores
    num_feats <-- number of features to display (display the num_feats top positive contributors according to composite scores).
                  if Set to 0, all featueres are displayed
    display_min <-- If true, display num_feats least (highest negatively) contributing features aswell
    display_composite <-- if true, display composite scores instead of individual models' attributions
    '''
    
    if visualise:
        plt.rcParams['figure.constrained_layout.use'] = True
    
    # Easier to run notebook cells asyncronously
    pub_data = deepcopy(pub_data)
    attributions = pub_data.attributions
    
    # Preprocess feature names
    for ind, row in pub_data.counterfactuals.iterrows():
        pub_data.counterfactuals.loc[ind] = add_units(pub_data.counterfactuals.loc[ind])
    
    pub_data.feature_values = add_units(pub_data.feature_values)
    
    if num_feats == 0:
        num_feats = len(attributions.columns)
    # Construct plot
    
    # Number of grid cells for table of counterfactuals to take up
    cellspertable = int(len(attributions.columns) / 2)
    
    # Number of grid cells for plot of single counterfactual to take up
    cellsperplot = 1
    
    # One for each feature/counterfactual and one for text
    if visualise:
        numfigs = num_feats * cellsperplot + 2
    else: 
        numfigs = ((num_feats+1)//2)
    
    if display_counterfactuals:
        numfigs += (cellspertable * len(pub_data.counterfactuals.index) + 1)
    
    # plt.rcParams['figure.figsize'] = [20, 3 * numfigs]
    
    fig = plt.figure(figsize=(21, 3 * numfigs))
    gs = gridspec.GridSpec(nrows=numfigs, ncols=2,
                        hspace=1, figure=fig)

    # fig.tight_layout()
    
    # Track y displacement in grid
    curr_figure = 0

    
    ###################### Textual explanation of classifier attributions ######################
    if not visualise:
        class_text_box = fig.add_subplot(gs[curr_figure:curr_figure + max((num_feats // 5), 1), :])
        curr_figure += max(((num_feats) // 5), 1)

        ordered_feats = list(reversed(pub_data.attributions.loc["COMPOSITE"].sort_values().index))

        text = temp_app_outcome.render(acceptance=pub_data.feature_values['Application Status'])
        text += temp_base_exp.render(num_feats=num_feats, 
                                    feature_vals=pub_data.feature_values,
                                   feats_by_contrib=ordered_feats,
                                    attributions=attributions.loc['COMPOSITE'],
                                    ordinals=[num2words(i, to="ordinal_num") for i in range(num_feats + 1)])

        class_text_box.text(-0.1, 1, text, wrap=True, linespacing=0.5)
        class_text_box.axis("off")
    
    ###################### Feature attribution plots ######################
    
    if visualise:
        class_text_box = fig.add_subplot(gs[curr_figure, :])
        curr_figure += 1
    
        text = temp_app_outcome.render(acceptance=pub_data.feature_values['Application Status'])

        class_text_box.text(-0.1, 1, text, wrap=True)
        class_text_box.axis("off")
    
        fp_text_box = fig.add_subplot(gs[curr_figure, :])
        curr_figure += 1

        # text explaining plots
        if num_feats == len(attributions.columns):
                text = """The following are the scores attributed to your application's features listed in order of importance:"""
        else:
            if display_min:
                text = "The following are the scores attributed to the " + str(num_feats) + """ features 
                of your application that contributed most/least to the classifier's decision: """
            else:
                if num_feats == 1:
                    text = "The following is the score attributed to the most imoportant feature of your application:"
                else:
                    text = "The following are the scores attributed to the " + str(num_feats) + """ most imoportant features of your application:"""

        fp_text_box.text(-0.1, 0.5, text, wrap=True)
        fp_text_box.axis("off")


        # Deide on common axis
        x_max = max([abs(max(attributions.max(axis=1))), 
                     abs(min(attributions.min(axis=1)))])

        xlim=(-1.1 *x_max, 1.1 * x_max)

        for i in range(num_feats):
            # find i-th most/least salient feature (by composite)
            feat_max = attributions.loc['COMPOSITE'].idxmax(axis=0)
            feat_min = attributions.loc['COMPOSITE'].idxmin(axis=0)

            if(display_composite):
                y_lab = ['SCORE']
                max_w = (attributions.loc['COMPOSITE'])[feat_max]
                min_w = (attributions.loc['COMPOSITE'])[feat_min]
            else:
                y_lab = list((pub_data.attributions.drop(['COMPOSITE'], axis=0)).index)
                max_w = [(attributions.loc[attr])[feat_max] for attr in y_lab]
                min_w = [(attributions.loc[attr])[feat_min] for attr in y_lab]

            # Plot i-th most/least salient feature

            colors_max = cm.autumn( [(1.0 / (i+1)) for i in range(len(y_lab))])
            colors_min = cm.winter( [(1.0 / (i+1)) for i in range(len(y_lab))])

            if display_min:
                axs1 = fig.add_subplot(gs[curr_figure : curr_figure + cellsperplot, 1])
                axs1.barh(y_lab, min_w, align='center', height=0.3, color=['#FFC20A' if val > 0 else '#0C7BDC' for val in max_w])
                axs1.set_xlim(xlim)
                axs1.set_xlabel('\% Of Total Feature Importance')
                axs1.set_title('Contribution of ' + feat_min + ' = ' + str(pub_data.feature_values[feat_min]))
                axs1.axvline(x=0, linewidth=1, color='k')

                axs2 = fig.add_subplot(gs[curr_figure : curr_figure + cellsperplot, 0])
                axs2.barh(y_lab, max_w, align='center', height=0.3, color=['#FFC20A' if val > 0 else '#0C7BDC' for val in min_w])
                axs2.set_xlim(xlim)
                axs1.xlabel('\% Of Total Feature Importance')
                axs2.set_title('Contribution of ' + feat_max + ' = ' + str(pub_data.feature_values[feat_max]))
                axs2.axvline(x=0, linewidth=1, color='k')

                # Remove features already displayed
                attributions = attributions.drop(feat_max, axis=1)
                attributions = attributions.drop(feat_min, axis=1)
            else:
                if type(max_w) != list:
                    max_w = [max_w]
                axs = fig.add_subplot(gs[curr_figure : curr_figure + cellsperplot, :])
                axs.barh(y_lab, max_w, color=['#FFC20A' if val > 0 else '#0C7BDC' for val in max_w])
                axs.set_xlim(xlim)
                axs.set_title('Contributions of ' + feat_max + ' = ' + str(pub_data.feature_values[feat_max]))
                axs.axvline(x=0, linewidth=1, color='k')
                axs.set_xlabel('\% Of Total Feature Importance')
                
                # Set bar labels
                ind = 0
                for i in axs.patches:
                    # get_width pulls left or right; get_y pushes up or down
                    axs.text(i.get_width()+.3, -1 * i.get_y(), \
                            str(round(max_w[ind], 2))+'%',
                            color='black')
                    
                    ind += 1
    
                
            attributions = attributions.drop(feat_max, axis=1)
            curr_figure += cellsperplot

    plt.rcParams['figure.constrained_layout.use'] = False
    
    if display_counterfactuals:
        # Display counterfactual explanations
        
        # Add counterfactual text
        text_box = fig.add_subplot(gs[curr_figure, :])
        curr_figure += 1
        
        text = "The following candidate profile/s similar to yours would have achieved the opposite outcome:"
        text_box.text(-0.1, 0.5, text, wrap=True)
        text_box.axis("off")
        
        cf_tables = []
        for index, row in pub_data.counterfactuals.iterrows():
            # Add table for each counterfactual explanation
            cf_tables.append(fig.add_subplot(gs[curr_figure :  curr_figure + cellspertable, :]))
            
            curr_figure += cellspertable
            
            cell_colours = [["#ffd1dc"] 
                            if row[attribute] != pub_data.feature_values[attribute] 
                            else ["#ffffff"]
                            for attribute in row.index]

            # cell_text = [[row[attribute]] for attribute in row.index]
            cell_text = [[ '\\textbf{' + str(pub_data.feature_values[attribute]) + '$\longrightarrow$ ' + str(row[attribute]) + '}']
                          if row[attribute] != pub_data.feature_values[attribute] 
                          else ['\\textbf{' + str(row[attribute]) + '}'] 
                          for attribute in row.index]
            
            tab = cf_tables[index].table(cellText=cell_text,
                                        rowLabels=['\\textbf{' + s + '}' for s in row.index],
                                        colLabels=['\\textbf{Value}'],
                                        loc="upper center",
                                        colWidths=[0.3],
                                        cellColours=cell_colours)
            
            tab.scale(1, 5)
            tab.set_fontsize(40)
            cf_tables[index].axis("off")
            pass
    return fig

In [ ]:
example_fig = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=1,
                      display_composite=False,
                      display_min=False,
                      display_counterfactuals=False)


# plt.savefig('explanation.png', bbox_inches='tight')

In [ ]:
# Render explanations to be used with experiments 

# This code can be uncommented for easy regeneration of plots
'''
text_few = construct_explanation(pub_data,
                      visualise=False,
                      num_feats=3,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)
text_few.savefig('survey/text_few/explanation.png', bbox_inches='tight', dpi=200)

chart_few = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=3,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)
chart_few.savefig('survey/chart_few/explanation.png', bbox_inches='tight', dpi=200)


chart_many = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=0,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)

chart_many.savefig('survey/chart_many/explanation.png', bbox_inches='tight', dpi=200)


chart_models = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=3,
                      display_composite=False,
                      display_min=False,
                      display_counterfactuals=False)
chart_models.savefig('survey/chart_few_models/explanation.png', bbox_inches='tight', dpi=200)



chart_count = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=3,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=True)
chart_count.savefig('survey/chart_few_count/explanation.svg', bbox_inches='tight', dpi=200)
'''
# Feature Quantity explanations


# CHART SINGLE
'''

example_fig = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=1,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)


plt.savefig('single_charts.png', bbox_inches='tight')
'''

'''

# CHART FEW

example_fig = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=3,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)


plt.savefig('three_charts.png', bbox_inches='tight')
'''

# CHART ALL

'''

example_fig = construct_explanation(pub_data,
                      visualise=True,
                      num_feats=0,
                      display_composite=True,
                      display_min=False,
                      display_counterfactuals=False)

plt.savefig('eleven_charts.png', bbox_inches='tight', dpi=200)
'''


In [ ]:
# Display table of feature values

fig = plt.figure(figsize=(6, 5), dpi=300)

cell_text = [[str(add_units(pub_data.feature_values)[attribute])] 
             for attribute in pub_data.feature_values.index]
    
f_value_table = plt.table(cellText=cell_text[:-1],
          rowLabels=pub_data.feature_values.index[:-1],
          colLabels=['Value'],
          loc="upper center",
          colWidths=[0.3])

f_value_table.scale(1, 3)
f_value_table.fontsize = 16

plt.axis('off')

# plt.savefig('survey/data_instance/feature_values.svg', bbox_inches='tight')

# INTERACTIVE DISPLAY TOOL

## An interactive widget that can be used to explore datapoints in the explanation space

In [ ]:


def f(display_model_scores, display_all_features, display_counterfactuals, textual_attributions):
    if display_all_features:
        nf = 0
    else:
        nf = 3
    fig = construct_explanation(pub_data,
                      num_feats=nf,
                      visualise=not textual_attributions,
                      display_composite=not display_model_scores,
                      display_min=False,
                      display_counterfactuals= display_counterfactuals)
    # fig.show()

interactive_plot = interactive(f, display_model_scores=False, display_all_features=False, display_counterfactuals=False, textual_attributions=False)
output = interactive_plot.children[-1]
#output.layout.height = '2000px'

interactive_plot